In [66]:
# библиотеки
import pandas as pd
import numpy as np
import re

In [67]:
work_link = {
    'elements': r'..\Bandgap prediction\Data\Elements\elements for ML.csv',
    'bandgap': r'D:..\Bandgap prediction\Data\Bandgap oxides and nitrides only.csv',
    'interaction data nitrides': r'..\Bandgap prediction\Data\interaction data nitrides.csv',
    'interaction data oxides': r'D:..\Bandgap prediction\Data\interaction data oxides.csv',
    'oxides and nitrides  for predict': r'D:..\Bandgap prediction\Data\oxides and nitrides  for predict.csv'
}

In [68]:
elements = pd.read_csv(filepath_or_buffer=work_link['elements'], index_col='Symbol')

In [69]:
compound = pd.read_csv(filepath_or_buffer=work_link['bandgap'], sep=';')

In [70]:
interaction_data_nitrides = pd.read_csv(filepath_or_buffer=work_link['interaction data nitrides'], sep=';',index_col='Example')

In [71]:
interaction_data_oxides = pd.read_csv(filepath_or_buffer=work_link['interaction data oxides'], sep=';', index_col='Example')

In [145]:
predicted_data = pd.read_csv(filepath_or_buffer=work_link['oxides and nitrides  for predict'], sep=';', index_col='Compound')

In [75]:
import collections
[i for i in collections.Counter(compound['Compound']).items() if i[1] > 1]

[('Fe2TiO5', 2),
 ('Fe2V4O13', 2),
 ('ZrMo2O8', 2),
 ('BaWO4', 2),
 ('BaMoO4', 2),
 ('MgIn2O4', 2),
 ('BiFeO3', 2),
 ('Ag3VO4', 2),
 ('ZnGeN2', 2)]

In [92]:
compound.Band_gap = compound.Band_gap.map(lambda x: float(re.sub(pattern=',', repl='.', string=x)))

In [95]:
# уникальные соединения
unique_index = pd.unique(compound['Compound'])

# создам df с уникальными соединениями. Занчения ширины запрещенной зоны для соединения у которого несколько показаний, усредняется
data = pd.DataFrame(data=[np.around(a=np.mean(index.values), decimals=4) for index in map(lambda x: compound[compound['Compound'] == x]['Band_gap'], unique_index)],
 columns=['Band_gap'], index=pd.Index(unique_index))
data.head()

,Band_gap
Mn2SiO4,3.25
Mn4SiO7,1.52
Mn7SiO12,3.15
AlSi2O5,9.10
Fe2SiO4,7.80


In [96]:
# регулярное выражение для разложения соединения
reg = r"(([A-Z](?![a-z])+|[A-Z][a-z]+)(\d+|))"
# re.findall(pattern=reg, string='Bi2Te2Se') -- > [('Bi2', 'Bi', '2'), ('Te2', 'Te', '2'), ('Se', 'Se', '')]

# расзбирает соединение 
def fun_dearch_elements(element):
    d = dict()
    s = re.findall(pattern=reg, string=element)
    for i in range(len(s)):
        d['element_{f_1}'.format(f_1=i)] = s[i][1]
        d['number_{f_1}'.format(f_1=i)] = s[i][2]
    return d

In [97]:
print(fun_dearch_elements('Mn2SiO4'))

{'element_0': 'Mn', 'number_0': '2', 'element_1': 'Si', 'number_1': '', 'element_2': 'O', 'number_2': '4'}


In [100]:
ele = [re.findall(pattern=reg, string=i)[j][1] for i in data.index for j in range(len(re.findall(pattern=reg, string=i)))]
print('Все элементы в соединениях: ', ele[:50])


Все элементы в соединениях:  ['Mn', 'Si', 'O', 'Mn', 'Si', 'O', 'Mn', 'Si', 'O', 'Al', 'Si', 'O', 'Fe', 'Si', 'O', 'Y', 'Si', 'O', 'Zr', 'Si', 'O', 'In', 'Si', 'O', 'Zn', 'Si', 'O', 'Mn', 'Co', 'O', 'Mn', 'Co', 'O', 'Mn', 'Al', 'O', 'Fe', 'Mn', 'O', 'Cr', 'Mn', 'O', 'Mn', 'Y', 'O', 'Mn', 'Sn', 'O', 'Ga', 'Mn']


In [103]:
dek_el = set(ele) - set(elements.index)
print('Элементы без данных: ', dek_el)

Элементы без данных:  {'La'}


In [104]:
# создам список соеинений, которые необходимо удвлить из data
del_compaunds = []
for compound in data.index:
    a = re.findall(pattern=reg, string=compound)
    b = [a[i][1] for i in range(len(a))]
    if not set(b).isdisjoint(dek_el):
        del_compaunds.append(compound)

In [105]:
print('Список соединений который необходимо удалить:{f1}'.format(f1=del_compaunds))

Список соединений который необходимо удалить:['LaFeO3', 'LaCoO3']


In [106]:
data.drop(del_compaunds, inplace=True)

In [108]:
elements.head()

,Atomic_Number,NUMBER_OF_Electrons_at_last_orbitale,NUMBER_OF_Electrons_at_before_last_orbitale,NUMBER_OF_electrones_at_last_level,NUMBER_OF_vacancies_at_outer_orbitale,Number_of_active_electrons_at_inner_level,Max_valency,Atomic_Mass,Electronegativity,Atomic_radius,Covalent_radius,Ionization_potential,Electron_affinity,Period,Group,Block
Symbol,,,,,,,,,,,,,,,,
Li,3,1,0,1,1,0,1,7.000000,0.98,145.0,134,5.392,59.6,2,0,0
B,5,1,2,3,5,0,1,10.810000,2.04,98.0,82,8.298,26.7,2,1,1
C,6,2,2,4,4,0,2,12.011000,2.55,77.0,77,11.260,153.9,2,2,1
N,7,3,2,5,3,0,5,14.006700,3.04,92.0,75,14.480,7.0,2,3,1
Na,11,1,0,2,1,0,1,22.989769,0.93,190.0,154,5.139,52.8,3,0,0


In [109]:
fun_dearch_elements('SnS2')

{'element_0': 'Sn', 'number_0': '', 'element_1': 'S', 'number_1': '2'}

In [125]:

for index in data.index:
    if len(re.findall(pattern=reg, string=index)) > 3 or len(re.findall(pattern=reg, string=index)) <3:
        print(True)

In [111]:
for element in elements.columns:
    data.insert(loc=0, column='{f_1}_1'.format(f_1=element),
                value=[elements.at[fun_dearch_elements(comp)['element_0'], element] for comp in data.index])

In [117]:
data.insert(loc=0, column='number_element_1',
            value=[fun_dearch_elements(comp)['number_0'] if fun_dearch_elements(comp)['number_0'] != '' else 1  for comp in data.index])

In [118]:
data.head()

,number_element_1,Block_1,Group_1,Period_1,Electron_affinity_1,Ionization_potential_1,Covalent_radius_1,Atomic_radius_1,Electronegativity_1,Atomic_Mass_1,Max_valency_1,Number_of_active_electrons_at_inner_level_1,NUMBER_OF_vacancies_at_outer_orbitale_1,NUMBER_OF_electrones_at_last_level_1,NUMBER_OF_Electrons_at_before_last_orbitale_1,NUMBER_OF_Electrons_at_last_orbitale_1,Atomic_Number_1,Band_gap
Mn2SiO4,2,2,10,4,0.0,7.434,139,127.0,1.55,54.938040,7,5,0,2,0,2,25,3.25
Mn4SiO7,4,2,10,4,0.0,7.434,139,127.0,1.55,54.938040,7,5,0,2,0,2,25,1.52
Mn7SiO12,7,2,10,4,0.0,7.434,139,127.0,1.55,54.938040,7,5,0,2,0,2,25,3.15
AlSi2O5,1,1,1,3,42.5,5.986,118,143.0,1.61,26.981538,1,0,5,3,2,1,13,9.10
Fe2SiO4,2,2,11,4,15.7,7.902,125,126.0,1.83,55.840000,6,6,0,2,0,2,26,7.80


In [119]:
# 2 элемент

for element in elements.columns:
    data.insert(loc=0, column='{f_1}_2'.format(f_1=element),
                value=[elements.at[fun_dearch_elements(comp)['element_1'], element] if 'element_1' in fun_dearch_elements(comp).keys() else 0 for comp in data.index])

In [120]:
data.insert(loc=0, column='number_element_2',
            value=[(lambda x: fun_dearch_elements(x)['number_1'] if fun_dearch_elements(x)['number_1'] != '' else 1)(comp)
            if 'number_1' in fun_dearch_elements(comp).keys() else 0  for comp in data.index])

In [121]:
data

,number_element_2,Block_2,Group_2,Period_2,Electron_affinity_2,Ionization_potential_2,Covalent_radius_2,Atomic_radius_2,Electronegativity_2,Atomic_Mass_2,...,Electronegativity_1,Atomic_Mass_1,Max_valency_1,Number_of_active_electrons_at_inner_level_1,NUMBER_OF_vacancies_at_outer_orbitale_1,NUMBER_OF_electrones_at_last_level_1,NUMBER_OF_Electrons_at_before_last_orbitale_1,NUMBER_OF_Electrons_at_last_orbitale_1,Atomic_Number_1,Band_gap
Mn2SiO4,1,1,2,3,133.6,8.152,111,132.0,1.90,28.08500,...,1.55,54.938040,7,5,0,2,0,2,25,3.25
Mn4SiO7,1,1,2,3,133.6,8.152,111,132.0,1.90,28.08500,...,1.55,54.938040,7,5,0,2,0,2,25,1.52
Mn7SiO12,1,1,2,3,133.6,8.152,111,132.0,1.90,28.08500,...,1.55,54.938040,7,5,0,2,0,2,25,3.15
AlSi2O5,2,1,2,3,133.6,8.152,111,132.0,1.90,28.08500,...,1.61,26.981538,1,0,5,3,2,1,13,9.10
Fe2SiO4,1,1,2,3,133.6,8.152,111,132.0,1.90,28.08500,...,1.83,55.840000,6,6,0,2,0,2,26,7.80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Ca2ZnN2,1,2,13,4,0.0,9.394,131,138.0,1.65,65.40000,...,1.00,40.080000,2,0,0,2,0,2,20,1.90
CaZn2N2,2,2,13,4,0.0,9.394,131,138.0,1.65,65.40000,...,1.00,40.080000,2,0,0,2,0,2,20,1.90
Li3ScN2,1,2,6,4,18.1,6.561,144,162.0,1.36,44.95591,...,0.98,7.000000,1,0,1,1,0,1,3,2.90
NaSnN,1,1,2,5,107.3,7.344,141,162.0,1.96,118.71000,...,0.93,22.989769,1,0,1,2,0,1,11,2.50


In [122]:
# 3 элемент

for element in elements.columns:
    data.insert(loc=0, column='{f_1}_3'.format(f_1=element),
                value=[elements.at[fun_dearch_elements(comp)['element_2'], element] if 'element_2' in fun_dearch_elements(comp).keys() else 0 for comp in data.index])

data.insert(loc=0, column='number_element_3',
            value=[(lambda x: fun_dearch_elements(x)['number_2'] if fun_dearch_elements(x)['number_2'] != '' else 1)(comp)
            if 'number_2' in fun_dearch_elements(comp).keys() else 0  for comp in data.index])

In [123]:
data

,number_element_3,Block_3,Group_3,Period_3,Electron_affinity_3,Ionization_potential_3,Covalent_radius_3,Atomic_radius_3,Electronegativity_3,Atomic_Mass_3,...,Electronegativity_1,Atomic_Mass_1,Max_valency_1,Number_of_active_electrons_at_inner_level_1,NUMBER_OF_vacancies_at_outer_orbitale_1,NUMBER_OF_electrones_at_last_level_1,NUMBER_OF_Electrons_at_before_last_orbitale_1,NUMBER_OF_Electrons_at_last_orbitale_1,Atomic_Number_1,Band_gap
Mn2SiO4,4,1,5,2,141.0,13.56,66,48.0,3.44,15.9994,...,1.55,54.938040,7,5,0,2,0,2,25,3.25
Mn4SiO7,7,1,5,2,141.0,13.56,66,48.0,3.44,15.9994,...,1.55,54.938040,7,5,0,2,0,2,25,1.52
Mn7SiO12,12,1,5,2,141.0,13.56,66,48.0,3.44,15.9994,...,1.55,54.938040,7,5,0,2,0,2,25,3.15
AlSi2O5,5,1,5,2,141.0,13.56,66,48.0,3.44,15.9994,...,1.61,26.981538,1,0,5,3,2,1,13,9.10
Fe2SiO4,4,1,5,2,141.0,13.56,66,48.0,3.44,15.9994,...,1.83,55.840000,6,6,0,2,0,2,26,7.80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Ca2ZnN2,2,1,3,2,7.0,14.48,75,92.0,3.04,14.0067,...,1.00,40.080000,2,0,0,2,0,2,20,1.90
CaZn2N2,2,1,3,2,7.0,14.48,75,92.0,3.04,14.0067,...,1.00,40.080000,2,0,0,2,0,2,20,1.90
Li3ScN2,2,1,3,2,7.0,14.48,75,92.0,3.04,14.0067,...,0.98,7.000000,1,0,1,1,0,1,3,2.90
NaSnN,1,1,3,2,7.0,14.48,75,92.0,3.04,14.0067,...,0.93,22.989769,1,0,1,2,0,1,11,2.50


In [146]:
for element in elements.columns:
    predicted_data.insert(loc=0, column='{f_1}_1'.format(f_1=element),
                value=[elements.at[fun_dearch_elements(comp)['element_0'], element] for comp in predicted_data.index])

predicted_data.insert(loc=0, column='number_element_1',
            value=[fun_dearch_elements(comp)['number_0'] if fun_dearch_elements(comp)['number_0'] != '' else 1  for comp in predicted_data.index])

In [149]:
# 2 элемент

for element in elements.columns:
    predicted_data.insert(loc=0, column='{f_1}_2'.format(f_1=element),
                value=[elements.at[fun_dearch_elements(comp)['element_1'], element]
                if 'element_1' in fun_dearch_elements(comp).keys() else 0 for comp in predicted_data.index])

predicted_data.insert(loc=0, column='number_element_2',
            value=[(lambda x: fun_dearch_elements(x)['number_1'] if fun_dearch_elements(x)['number_1'] != '' else 1)(comp)
            if 'number_1' in fun_dearch_elements(comp).keys() else 0  for comp in predicted_data.index])

In [150]:
# 3 элемент

for element in elements.columns:
    predicted_data.insert(loc=0, column='{f_1}_3'.format(f_1=element),
                value=[elements.at[fun_dearch_elements(comp)['element_2'], element]
                if 'element_2' in fun_dearch_elements(comp).keys() else 0 for comp in predicted_data.index])

predicted_data.insert(loc=0, column='number_element_3',
            value=[(lambda x: fun_dearch_elements(x)['number_2'] if fun_dearch_elements(x)['number_2'] != '' else 1)(comp)
            if 'number_2' in fun_dearch_elements(comp).keys() else 0  for comp in predicted_data.index])

In [154]:
interaction_data_nitrides['X-Y'].unique()

array([0], dtype=int64)

In [161]:
data['Atomic_Number_3'].unique()

array([ 8, 53, 16,  7], dtype=int64)

In [173]:
data_oxide = data[data['Atomic_Number_3'] == 8]
target_oxide = data_oxide['Band_gap']
data_oxide.drop(columns='Band_gap', inplace=True)

In [176]:
data_nitride = data[data['Atomic_Number_3'] == 7]
target_nitride = data_nitride['Band_gap']
data_nitride.drop(columns='Band_gap', inplace=True)

In [230]:
# all_interaction_data = list(interaction_data_nitrides.index) + list(interaction_data_oxides.index)

In [236]:
# all_compound = list(data.index) + list(predicted_data.index)

In [237]:
# set(all_compound) - set(all_interaction_data)

{'Ag3PO4',
 'Ag3VO4',
 'AgBiO3',
 'AgCoO2',
 'AgGaO2',
 'AgNbO3',
 'AgTaO3',
 'AgVO3',
 'Al2CoN3',
 'Al2CrN3',
 'Al2CuO4',
 'Al2FeN3',
 'Al2FeO4',
 'Al2ZnO4',
 'Al3GaN4',
 'Al5BiN4',
 'AlCoN2',
 'AlCr2N3',
 'AlCrN2',
 'AlCuO2',
 'AlFe2N3',
 'AlFeN2',
 'AlGa3N4',
 'AlGaN2',
 'AlInN2',
 'AlNbO4',
 'AlVO4',
 'AlYO3',
 'AlZn35O36',
 'Ba2MnO3',
 'Ba2ZnN2',
 'Ba3Mn2O8',
 'BaAl2O4',
 'BaFe4O7',
 'BaMn2O3',
 'BaMn2O8',
 'BaMnO3',
 'BaMoO4',
 'BaSb2O6',
 'BaSi7N10',
 'BaSiN2',
 'BaSnO3',
 'BaTiO3',
 'BaWO4',
 'BaZnO2',
 'BaZrO3',
 'BaZrS3',
 'Bi25FeO40',
 'Bi2MoO6',
 'Bi2WO6',
 'Bi4Ti3O12',
 'BiFeO3',
 'BiOI',
 'BiPO4',
 'BiVO4',
 'Ca2Mn2O5',
 'Ca2Mn3O8',
 'Ca2Nb2O7',
 'Ca2Si5N8',
 'Ca2SiO4',
 'Ca2Ta2O7',
 'Ca2ZnN2',
 'Ca3B2N4',
 'Ca4Mn2O7',
 'CaCO3',
 'CaGa2O4',
 'CaMg2N2',
 'CaMnO3',
 'CaMoO4',
 'CaSb2O6',
 'CaSiN2',
 'CaSnO3',
 'CaTiO3',
 'CaV2O6',
 'CaWO4',
 'CaZn2N2',
 'Cd2Ge7O16',
 'Cd2GeO4',
 'Cd2SnO4',
 'CdGa2O4',
 'CdIn2O4',
 'CdMoO4',
 'CdSb2O6',
 'CdSnO3',
 'CdTiO3',
 'CdV2O4',
 'CdW

In [187]:
# model
from sklearn.model_selection import train_test_split
X_train_oxide, X_test_oxide, y_train_oxide, y_test_oxide = train_test_split(data_oxide, target_oxide, random_state=20)

In [190]:
X_train_nitride, X_test_nitride, y_train_nitride, y_test_nitride = train_test_split(data_nitride, target_nitride, random_state=20)

In [195]:
X_train = pd.concat([X_train_nitride, X_train_oxide])
y_train = pd.concat([y_train_nitride, y_test_oxide])

In [196]:
X_test = pd.concat([X_test_nitride, X_test_oxide])
y_test = pd.concat([y_test_nitride, y_test_oxide])